# LS88 Project

###   Imports

In [45]:
%matplotlib inline
import numpy as np
import datascience as ds
from datascience import Table, make_array, predicates
import nba_py as py
from nba_py import player
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

### Load the Main Table

In [142]:
main = Table().read_table('PbP_15_16_reduced.csv', sep=',')
main.show(5)

GAME_ID,EVENTNUM,PCTIMESTRING,TIME,PERIOD,HOME_TEAM,AWAY_TEAM,HOME_SCORE,AWAY_SCORE,POINTS_SCORED,SHOT_MADE,SHOT_TYPE,SHOT_DIST,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_NICKNAME,ASSIST_PLAYER_ID,EVENTMSGTYPE,EVENTMSGACTIONTYPE
21500001,0,12:00,0,1,Hawks,Pistons,0,0,0,nan,nan,nan,0,nan,nan,nan,Start Period,nan
21500001,1,12:00,0,1,Hawks,Pistons,0,0,0,nan,nan,nan,201143,Al Horford,Hawks,nan,Jumpball,nan
21500001,2,11:41,19,1,Hawks,Pistons,0,0,0,0,Layup,2,203083,Andre Drummond,Pistons,nan,Miss,Driving Layup
21500001,3,11:39,21,1,Hawks,Pistons,0,0,0,nan,nan,nan,203145,Kent Bazemore,Hawks,nan,Rebound,Player
21500001,4,11:37,23,1,Hawks,Pistons,0,0,0,nan,nan,nan,203145,Kent Bazemore,Hawks,nan,Turnover,Out of Bounds - Bad Pass Turnover


### Helper Functions

In [40]:
def calc_efg(FGM, THREEPTM, FGA):
    return (FGM + (0.5 * THREEPTM)) / FGA

def calc_ts(FTA, PTS, FGA):
    return PTS / ((2 * FGA) + (0.88 * FTA))

def convert_season_to_index(season):
    end = season[5:]
    int_year = int(end)
    return 18 - int_year

def time_string_to_number(time_string):
    index = len(time_string)-3
    minute = int(time_string[:index:])
    seconds = int(time_string[index::].replace(':', '')) / 60
    return minute + seconds

### Get Any Player's TS and ES

In [229]:
def get_season_stats(first, last):
    first = first.replace('.', '')
    print(first)
    print(last)
        
    player_id = py.player.get_player(first, last_name=last)
    
    print('HELLO')
          
    player_general_splits = py.player.PlayerGeneralSplits(player_id)

    player_yearoveryear_splits = py.player.PlayerYearOverYearSplits(player_id, '2015-16')

    stats = ds.Table.from_df(player_yearoveryear_splits.by_year())
    
    stats = stats.where('GROUP_VALUE', '2015-16')
    
    if stats.num_rows < 1:
        return []

    FGA = stats.column('FGA').item(0)
    FG3M = stats.column('FG3M').item(0)
    FGM = stats.column('FGM').item(0)

    EFG = calc_efg(FGM, FG3M, FGA)
    
    FTA = stats.column('FTA').item(0)
    PTS = stats.column('PTS').item(0)
    
    TS = calc_ts(FTA, PTS, FGA)

    return [TS, EFG]


In [228]:
get_season_stats('Kevin', 'Durant')

Kevin
Durant
HELLO


KeyboardInterrupt: 

### Creates a Table of Close Games Only

In [42]:
# Takes in a csv filepath of one of the EightThirtyFour data sets
# and filters the data to games with a 10 or less point lead in 
# the last 6 minutes of the game.
def game_filter(csv_file): 
    
    
    pbp = Table().read_table(csv_file)
        
    last_quarter = pbp.where('PERIOD', predicates.are.equal_to(4))
    transformed_minutes = last_quarter.apply(time_string_to_number, 'PCTIMESTRING')
    last_quarter_and_minutes = last_quarter.with_column('TIME', transformed_minutes)
    
    find_close_scores = last_quarter_and_minutes.where('TIME', predicates.are.between(6, 6.5))  

    close_games_ids = make_array()   
    last_id = 0
    
    for i in np.arange(find_close_scores.num_rows):      
        row = find_close_scores.row(i)    
        current_id = int(row[0])
        if current_id != last_id:
            diff = abs(int(row[7]) - int(row[8]))
            if diff <= 10:
                close_games_ids = np.append(current_id, close_games_ids)
        last_id = current_id
        
        
    close_games_table = Table().with_column('Close Games', close_games_ids)    
    time_less_than_six = last_quarter_and_minutes.where('TIME', predicates.are.below(6.6))
    time_less_than_six = time_less_than_six.join('GAME_ID', close_games_table, 'Close Games')
    
    return time_less_than_six


### Writes the New Table to File

In [53]:
#Write to File
game_filter('PbP_15_16_reduced.csv').to_csv('PbP_15_16_trimmed.csv')

In [139]:
games = Table().read_table('PbP_15_16_trimmed.csv', sep=',')
games

GAME_ID,EVENTNUM,PCTIMESTRING,TIME,PERIOD,HOME_TEAM,AWAY_TEAM,HOME_SCORE,AWAY_SCORE,POINTS_SCORED,SHOT_MADE,SHOT_TYPE,SHOT_DIST,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_NICKNAME,ASSIST_PLAYER_ID,EVENTMSGTYPE,EVENTMSGACTIONTYPE
21500002,474,6:32,6.53333,4,Bulls,Cavaliers,80,79,0,nan,nan,nan,2747,J.R. Smith,Cavaliers,nan,Personal Foul,P.FOUL
21500002,476,6:20,6.33333,4,Bulls,Cavaliers,82,79,2,1,Shot,15,201565,Derrick Rose,Bulls,nan,Make,Pullup Jump Shot
21500002,479,6:01,6.01667,4,Bulls,Cavaliers,82,81,2,1,Dunk,nan,202684,Tristan Thompson,Cavaliers,2590,Make,Dunk
21500002,480,5:50,5.83333,4,Bulls,Cavaliers,82,81,0,0,Layup,2,201565,Derrick Rose,Bulls,nan,Miss,Driving Layup
21500002,481,5:48,5.8,4,Bulls,Cavaliers,82,81,0,nan,nan,nan,201567,Kevin Love,Cavaliers,nan,Rebound,Player
21500002,482,5:32,5.53333,4,Bulls,Cavaliers,82,83,2,1,Shot,7,2544,LeBron James,Cavaliers,nan,Make,Driving Floating Jump Shot
21500002,483,5:18,5.3,4,Bulls,Cavaliers,84,83,2,1,Shot,13,202703,Nikola Mirotic,Bulls,nan,Make,Driving Floating Jump Shot
21500002,484,5:18,5.3,4,Bulls,Cavaliers,84,83,0,nan,nan,nan,202684,Tristan Thompson,Cavaliers,nan,Personal Foul,S.FOUL
21500002,485,5:18,5.3,4,Bulls,Cavaliers,85,83,1,nan,nan,nan,202703,Nikola Mirotic,Bulls,nan,Free Throw,Free Throw 1 of 1
21500002,488,5:07,5.11667,4,Bulls,Cavaliers,85,83,0,0,Layup,3,201567,Kevin Love,Cavaliers,nan,Miss,Reverse Layup


### Helper Funciton to Create Game Reference Table

In [107]:
# Sort the trimmed table to analyze on a game by game basis
# builds a list of tuples of indecies, each tubple represents a unquue game in the table

def isolate_game(table):
    games = []
    row_start = table.row(0)  
    last_id1 = int(row_start[0])
    current_id1 = 0
    last_index = 0
    count = 0
    
    for i in np.arange(table.num_rows):   
        
        row = table.row(i)           
        current_id1 = int(row[0])
        
        if current_id1 != last_id1:
            x = [last_index, i-1]

            games.append(x)
            count += 1
            
            # reset indicies
            last_index = i
            last_id1 = current_id1
        
    return games

games_list = isolate_game(games)    

    

### Creates Game Reference Table

In [119]:
def construct_game_reference_table():
    count = 1
    season = Table(make_array('HOME_TEAM', 'AWAY_TEAM'))
    array = make_array()
    for t in games_list:
        index = t[0]
        row = games.take(index).select('HOME_TEAM', 'AWAY_TEAM')
        array = np.append(array, count)
        count += 1
        season = season.with_row(row)
    season = season.with_column('GAME NUMBER', array).with_column('INDICIES', games_list)
    return season 
      
construct_game_reference_table()       

HOME_TEAM,AWAY_TEAM,GAME NUMBER,INDICIES
Bulls,Cavaliers,1,[ 0 69]
Magic,Wizards,2,[ 70 149]
Celtics,76ers,3,[150 224]
Pistons,Jazz,4,[225 301]
Pacers,Raptors,5,[302 396]
Thunder,Spurs,6,[397 475]
Clippers,Kings,7,[476 545]
Lakers,Timberwolves,8,[546 624]
Pacers,Grizzlies,9,[625 690]
Thunder,Magic,10,[691 773]


### Creates Player Reference Table

In [230]:
players = games.select('PLAYER1_ID', 'PLAYER1_NAME')
players = players.group('PLAYER1_ID', max).where('PLAYER1_NAME max', predicates.are.not_equal_to('nan')).relabel(1, 'PLAYER_NAME')

def first(string):
    name = string.split(' ')
    first = name[0]
    return first
    
def last(string):
    name = string.split(' ')
    if len(name) == 1:
        return 'nan'
    first = name[1]
    return first

players = players.with_column('FIRST_NAME', players.apply(first, 'PLAYER_NAME'))
players = players.with_column('LAST_NAME', players.apply(last, 'PLAYER_NAME'))


game1 = games.take(np.arange(0, 69))
names = game1.select('PLAYER1_NAME')

# players = players.join('PLAYER_NAME', names, 'PLAYER1_NAME').group('PLAYER1_ID', max)
players

# stats = make_array()
# for i in np.arange(0, players.num_rows):
#     row = players.row(i)
#     stats = np.append(get_season_stats(row[2], row[3]), stats)
    
# stats

PLAYER1_ID,PLAYER_NAME,FIRST_NAME,LAST_NAME
378,Craig Ehlo,Craig,Ehlo
380,Tony Smith,Tony,Smith
406,Shaquille O'Neal,Shaquille,O'Neal
416,Willie Burton,Willie,Burton
417,Sam Mitchell,Sam,Mitchell
426,Terry Davis,Terry,Davis
428,Sherman Douglas,Sherman,Douglas
431,Shawn Kemp,Shawn,Kemp
433,Buck Williams,Buck,Williams
434,Tony Dumas,Tony,Dumas


# Analysis Begins Here

In [ ]:
def shots_all_players(table):
    #This function finds the number of shots that each player takes in time period
    #(it doesn't distinguish the type of shot)
    #it can be used for an entire game or for the last 6 minutes
    shots = table.where('EVENTMSGTYPE', predicates.are.containing('M'))
    players = shots.select('EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PLAYER1_NAME', 'PLAYER1_TEAM_NICKNAME')
    count = players.group(('PLAYER1_NAME', 'PLAYER1_TEAM_NICKNAME'))
    return count

def max_shot_player_each_team(table, team1, team2):
    #This takes in a table like player_count above
    #It returns the player who shoots the most for each team
    player_team_one = table.where('PLAYER1_TEAM_NICKNAME', team1).sort('count', descending=True).column('PLAYER1_NAME').item(0)
    player_team_two = table.where('PLAYER1_TEAM_NICKNAME', team2).sort('count', descending=True).column('PLAYER1_NAME').item(0)
    return (player_team_one, player_team_two)